## Tutorial 01 - Command Line Interface

In [1]:
from typing import Dict
import os
import json
from pdb import set_trace

In [20]:
config = {
    'inputdir': '/afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/PUBHL2022/20221018_proj_all/',
    # edit your custom output directory here
    'outputdir': "${hh_combination_fw_path}/tutorials/projection2022/output/20221018_proj_all/",
    'resonant_type': 'nonres',
    'channels': ['bbyy', 'bbtautau', 'bbbb'],
    'lumi_scenarios': [1000, 1500, 2000, 2500, 3000],
    'syst_scenarios': ['stat_only', 'theo_exp_baseline', 'theo_only', 'run2_syst', 'optimistic'],
    'scenarios': {
        'SM_mu': {
            'file_expr': '<mass[F]>', 
            'config': '${hh_combination_fw_path}/configs/task_options/projection2022/proj_nonres_sm.yaml',
            'scheme': '${hh_combination_fw_path}/configs/correlation_schemes/projection2022/nonres_kl_v14.json',
            #options for baseline
            'minimizer_options_1': '${hh_combination_fw_path}/configs/minimizer_options/projection2021/projection_no_kl_unc.json',
            #options for theo_only, run2_syst
            'minimizer_options_2': None,
            'tasks': {'limit': True, 'likelihood': False, 'pvalue': True}},
        'SM_xsec': {
            'file_expr': '<mass[F]>', 
            'config': '${hh_combination_fw_path}/configs/task_options/projection2022/proj_nonres_sm.yaml',
            'scheme': '${hh_combination_fw_path}/configs/correlation_schemes/projection2022/nonres_kl_v14.json',
            #options for baseline
            'minimizer_options_1': '${hh_combination_fw_path}/configs/minimizer_options/projection2021/projection_fix_xs_uncertainty.json',
            #options for theo_only, run2_syst
            'minimizer_options_2': None,
            'tasks': {'limit': True, 'likelihood': False, 'pvalue': True}},
        'kl_individual': {
            'file_expr': '<mass[F]>_kl_<klambda[P]>',
            'config': '${hh_combination_fw_path}/configs/task_options/projection2022/proj_nonres_sm.yaml',
            'scheme': '${hh_combination_fw_path}/configs/correlation_schemes/projection2022/nonres_kl_v14.json',
            #options for baseline
            'minimizer_options_1': '${hh_combination_fw_path}/configs/minimizer_options/projection2021/projection_fix_xs_uncertainty_no_kl_unc.json',
            #options for theo_only, run2_syst
            'minimizer_options_2': '${hh_combination_fw_path}/configs/minimizer_options/projection2021/projection_fix_xs_uncertainty.json',
            'tasks': {'limit': True, 'likelihood': False, 'pvalue': False}},
        'kl_parameterised': {
            'file_expr': '<mass[F]>_kl',
            'param_expr': 'klambda=-2_8_0.1',
            'config': '${hh_combination_fw_path}/configs/task_options/projection2022/proj_nonres_kl.yaml',
            'scheme': '${hh_combination_fw_path}/configs/correlation_schemes/projection2022/nonres_kl_v14.json',
            #options for baseline
            'minimizer_options_1': '${hh_combination_fw_path}/configs/minimizer_options/projection2021/projection_fix_xs_uncertainty_no_kl_unc.json',
            #options for theo_only, run2_syst
            'minimizer_options_2': '${hh_combination_fw_path}/configs/minimizer_options/projection2021/projection_fix_xs_uncertainty.json',
            'tasks': {'limit': True, 'likelihood': True, 'pvalue': True}
        }
    },
    # special case for stat-only sysematics scenario
    'stat_only_minimizer_options': '${hh_combination_fw_path}/configs/minimizer_options/projection2021/projection_stat_only.json'
}

In [23]:
def get_commands(config:Dict, command_out:str='commands.sh'):
    channels = ",".join(config['channels'])
    resonant_type = config['resonant_type']
    inputdir = config['inputdir']
    outdir = config['outputdir']
    process_channels_cmd_str_list = []
    combine_ws_cmd_str_list = []
    comment_str_list = []
    for lumi_scenario in config['lumi_scenarios']:
        for syst_scenario in config['syst_scenarios']:
            if (syst_scenario == 'optimistic' and lumi_scenario != 3000):
                continue
            input_path = os.path.join(inputdir, f'lumi{lumi_scenario}ifb', syst_scenario)
            print("=========================================================")
            print(f"Luminosity Scenario: {lumi_scenario}")
            print(f"Systematics Scenario: {syst_scenario}")
            print("=========================================================")
            for task_scenario in config['scenarios']:
                if (syst_scenario == 'optimistic' and task_scenario != 'SM_mu'):
                    continue
                # only do SM limit and pvalue for lumi != 3000ifb
                if (lumi_scenario != 3000) and  (task_scenario != 'SM_mu'):
                    continue
                settings = config['scenarios'][task_scenario]
                output_path = os.path.join(outdir, f'lumi{lumi_scenario}ifb', syst_scenario, task_scenario)
                file_expr = settings['file_expr']
                param_expr = settings.get('param_expr', None)
                config_path = settings['config']
                correlation_scheme = settings['scheme']
                if syst_scenario == 'stat_only':
                    minimizer_options = config['stat_only_minimizer_options']
                elif syst_scenario == 'theo_exp_baseline':
                    minimizer_options = settings.get('minimizer_options_1', None)
                elif syst_scenario in ['theo_only', 'run2_syst']:
                    minimizer_options = settings.get('minimizer_options_2', None)
                elif syst_scenario == 'optimistic':
                    minimizer_options = settings.get('minimizer_options_1', None)
                else:
                    raise RuntimeError(f"unknown systematics scenario `{syst_scenario}`")
                base_cmd_str = f"-c {channels} -r {resonant_type} --file_expr \"{file_expr}\" --config {config_path}"
                if param_expr is not None:
                    base_cmd_str += f" --param_expr \"{param_expr}\""
                if minimizer_options is not None:
                    base_cmd_str += f" --minimizer_options {minimizer_options}"
                for task_name, flag in settings['tasks'].items():
                    if flag:
                        base_cmd_str += f" --do-{task_name}"
                    else:
                        base_cmd_str += f" --skip-{task_name}"
                process_channels_cmd_str = f"HHComb process_channels -i {input_path} -o {output_path} {base_cmd_str}"
                combine_ws_cmd_str = f"HHComb combine_ws -i {output_path} -s {correlation_scheme} {base_cmd_str}"
                print("")
                print(f"Task Scenario: {task_scenario}")
                print("")
                print(process_channels_cmd_str)
                print("")
                print(combine_ws_cmd_str)
                print("")
                comment_str_list.append(f"# lumi = {lumi_scenario}, syst = {syst_scenario}, task_scenario = {task_scenario}\n\n")
                process_channels_cmd_str_list.append(process_channels_cmd_str+"\n\n")
                combine_ws_cmd_str_list.append(combine_ws_cmd_str+"\n\n")
    if command_out is not None:
        f = open(command_out, 'w')
        for comment_str, process_channels_cmd_str in zip(comment_str_list, process_channels_cmd_str_list):
            f.write(comment_str)
            f.write(process_channels_cmd_str)
        for comment_str, combine_ws_cmd_str in zip(comment_str_list, combine_ws_cmd_str_list):
            f.write(comment_str)
            f.write(combine_ws_cmd_str)
    else:
        f = None

In [24]:
get_commands(config, command_out='commands.sh')

Luminosity Scenario: 1000
Systematics Scenario: stat_only

Task Scenario: SM_mu

HHComb process_channels -i /afs/cern.ch/work/z/zhangr/HHcomb/hh_combination_fw/hh_combination_fw/FullRun2Workspaces/original/PUBHL2022/20221018_proj_all/lumi1000ifb/stat_only -o ${hh_combination_fw_path}/tutorials/projection2022/output/20221018_proj_all/lumi1000ifb/stat_only/SM_mu -c bbyy,bbtautau,bbbb -r nonres --file_expr "<mass[F]>" --config ${hh_combination_fw_path}/configs/task_options/projection2022/proj_nonres_sm.yaml --minimizer_options ${hh_combination_fw_path}/configs/minimizer_options/projection2021/projection_stat_only.json --do-limit --skip-likelihood --do-pvalue

HHComb combine_ws -i ${hh_combination_fw_path}/tutorials/projection2022/output/20221018_proj_all/lumi1000ifb/stat_only/SM_mu -s ${hh_combination_fw_path}/configs/correlation_schemes/projection2022/nonres_kl_v14.json -c bbyy,bbtautau,bbbb -r nonres --file_expr "<mass[F]>" --config ${hh_combination_fw_path}/configs/task_options/project